# SQL query from table names - Continued

In [3]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [16]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [24]:
context = []
context.append({'role': 'system', 'content': """
-- Table: Employees
CREATE TABLE employees (
  ID_usr INTEGER PRIMARY KEY,
  name TEXT
)
-- Sample Rows:
-- (1, 'Alice Johnson')
-- (2, 'Bob Smith')
-- (3, 'Carol Davis')

-- Table: Salary
CREATE TABLE salary (
  ID_usr INTEGER,
  year DATE,
  salary REAL,
  PRIMARY KEY (ID_usr, year)
)
-- Sample Rows:
-- (1, '2023-01-01', 50000.00)
-- (2, '2023-01-01', 60000.00)
-- (3, '2023-01-01', 45000.00)

-- Table: Studies
CREATE TABLE studies (
  ID INTEGER PRIMARY KEY,
  ID_usr INTEGER,
  educational_level INTEGER,
  Institution TEXT,
  Years DATE,
  Speciality TEXT
)
-- Sample Rows:
-- (1, 1, 3, 'XYZ University', '2020-2024', 'Computer Science')
-- (2, 2, 2, 'ABC University', '2019-2023', 'Business Administration')
-- (3, 3, 1, 'LMN University', '2018-2022', 'Engineering')
"""})

In [25]:
#FEW SHOT SAMPLES

context.append({'role': 'system', 'content': """
-- Maintain the SQL order simple and efficient as you can, using valid SQLite, answer the following questions for the tables provided above.

Q: List all employee names.
A: SELECT name FROM employees

Q: What is Bob Smith's salary for the year 2023?
A: SELECT salary FROM salary
JOIN employees ON salary.ID_usr = employees.ID_usr
WHERE employees.name = 'Bob Smith' AND salary.year = '2023-01-01'

Q: Show all employees who studied at 'XYZ University'.
A: SELECT employees.name FROM employees
JOIN studies ON employees.ID_usr = studies.ID_usr
WHERE studies.Institution = 'XYZ University'

Q: How many employees have a salary above 50,000?
A: SELECT COUNT(*) FROM salary
WHERE salary > 50000

Q: Find all educational levels of employees who studied 'Engineering'.
A: SELECT DISTINCT educational_level FROM studies
WHERE Speciality = 'Engineering'
                
"""})

In [19]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [20]:
#new
context_user = context.copy()
print(return_CCRMSQL("Who is the employee with the highest salary?", context_user))

```sql
SELECT employees.name
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr
ORDER BY salary.salary DESC
LIMIT 1
```


In [21]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Who is the employee with the highest salary?", old_context_user))

This is your SQL:
```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query selects the name and salary of the employee with the highest salary by joining the "employees" and "salary" tables on the ID_usr column, ordering the results by salary in descending order, and limiting the output to only the top result.


In [22]:
#new
print(return_CCRMSQL("HWhat is the institution with the highest average salary for its graduates?", context_user))

```sql
SELECT Institution, AVG(salary.salary) AS avg_salary
FROM studies
JOIN salary ON studies.ID_usr = salary.ID_usr
GROUP BY Institution
ORDER BY avg_salary DESC
LIMIT 1;
```


In [23]:
#old
print(return_CCRMSQL("What is the institution with the highest average salary for its graduates?", old_context_user))

This is your SQL:
```sql
SELECT s.Institution, AVG(sa.salary) AS avg_salary
FROM studies s
JOIN employees e ON s.ID_usr = e.ID_usr
JOIN salary sa ON s.ID_usr = sa.ID_usr
GROUP BY s.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query joins the tables `studies`, `employees`, and `salary` on the user ID to calculate the average salary for graduates of each institution. It then selects the institution with the highest average salary for its graduates by ordering the results in descending order and limiting the output to the top result.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [29]:
context1 = []
context1.append({'role': 'system', 'content': """
You are a SQL assistant. Answer the question by writing the SQL command only.
Keep it simple and correct. Your database has Employees, Salary, and Studies tables.
                 
-- Table: Employees
CREATE TABLE employees (
  ID_usr INTEGER PRIMARY KEY,
  name TEXT
)
-- Sample Rows:
-- (1, 'Alice Johnson')
-- (2, 'Bob Smith')
-- (3, 'Carol Davis')

-- Table: Salary
CREATE TABLE salary (
  ID_usr INTEGER,
  year DATE,
  salary REAL,
  PRIMARY KEY (ID_usr, year)
)
-- Sample Rows:
-- (1, '2023-01-01', 50000.00)
-- (2, '2023-01-01', 60000.00)
-- (3, '2023-01-01', 45000.00)

-- Table: Studies
CREATE TABLE studies (
  ID INTEGER PRIMARY KEY,
  ID_usr INTEGER,
  educational_level INTEGER,
  Institution TEXT,
  Years DATE,
  Speciality TEXT
)
-- Sample Rows:
-- (1, 1, 3, 'XYZ University', '2020-2024', 'Computer Science')
-- (2, 2, 2, 'ABC University', '2019-2023', 'Business Administration')
-- (3, 3, 1, 'LMN University', '2018-2022', 'Engineering')

Q: Find the names of customers who ordered a product priced more than $500.
A: 
```sql
SELECT DISTINCT Customers.name
FROM Customers
JOIN Orders ON Customers.customer_id = Orders.customer_id
JOIN Products ON Orders.product_id = Products.product_id
WHERE Products.price > 500
```
This SQL query retrieves the names of customers who have ordered products priced over $500. 
It joins the Customers, Orders, and Products tables to filter based on product price.
"""})


In [34]:
print(return_CCRMSQL("How many employees have a salary greater than 50,000", context1))

```sql
SELECT COUNT(*)
FROM Employees
JOIN Salary ON Employees.ID_usr = Salary.ID_usr
WHERE Salary.salary > 50000
```


In [32]:
context2 = []
context2.append({'role': 'system', 'content': """
You are a helpful SQL assistant. Based on the following schema, write clean, valid SQLite code to answer the question.
Return the SQL query inside a code block, followed by a short explanation in plain English.

-- Table: Employees
CREATE TABLE employees (
  ID_usr INTEGER PRIMARY KEY,
  name TEXT
)
-- Sample Rows:
-- (1, 'Alice Johnson')
-- (2, 'Bob Smith')
-- (3, 'Carol Davis')

-- Table: Salary
CREATE TABLE salary (
  ID_usr INTEGER,
  year DATE,
  salary REAL,
  PRIMARY KEY (ID_usr, year)
)
-- Sample Rows:
-- (1, '2023-01-01', 50000.00)
-- (2, '2023-01-01', 60000.00)
-- (3, '2023-01-01', 45000.00)

-- Table: Studies
CREATE TABLE studies (
  ID INTEGER PRIMARY KEY,
  ID_usr INTEGER,
  educational_level INTEGER,
  Institution TEXT,
  Years DATE,
  Speciality TEXT
)
-- Sample Rows:
-- (1, 1, 3, 'XYZ University', '2020-2024', 'Computer Science')
-- (2, 2, 2, 'ABC University', '2019-2023', 'Business Administration')
-- (3, 3, 1, 'LMN University', '2018-2022', 'Engineering')

Q: Who is the employee with the highest salary?
A:
```sql
SELECT employees.name
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr
ORDER BY salary.salary DESC
LIMIT 1"
```
This SQL query retrieves the name of the employee with the highest salary by joining the employees and salary tables, ordering by salary in descending order, and limiting the result to one row."
"""
})


In [35]:
print(return_CCRMSQL("How many employees have a salary greater than 50,000", context2))

```sql
SELECT COUNT(*)
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr
WHERE salary.salary > 50000
```

This SQL query counts the number of employees who have a salary greater than 50,000 by joining the employees and salary tables, filtering the results to include only those with a salary greater than 50,000.


In [41]:
context3 = []
context3.append({'role': 'system', 'content': """
You are a helpful SQL assistant. Based on the following schema, write clean, valid SQLite code to answer the question.
Return the SQL query inside a code block, followed by a short explanation in plain English. 
provide a sample result with the query.
If the user asks questions that are not related to SQL or do not match the schema provided, 
kindly inform them that you can only assist with SQL-related queries.

-- Table: Employees
CREATE TABLE employees (
  ID_usr INTEGER PRIMARY KEY,
  name TEXT
)
-- Sample Rows:
-- (1, 'Alice Johnson')
-- (2, 'Bob Smith')
-- (3, 'Carol Davis')

-- Table: Salary
CREATE TABLE salary (
  ID_usr INTEGER,
  year DATE,
  salary REAL,
  PRIMARY KEY (ID_usr, year)
)
-- Sample Rows:
-- (1, '2023-01-01', 50000.00)
-- (2, '2023-01-01', 60000.00)
-- (3, '2023-01-01', 45000.00)

-- Table: Studies
CREATE TABLE studies (
  ID INTEGER PRIMARY KEY,
  ID_usr INTEGER,
  educational_level INTEGER,
  Institution TEXT,
  Years DATE,
  Speciality TEXT
)
-- Sample Rows:
-- (1, 1, 3, 'XYZ University', '2020-2024', 'Computer Science')
-- (2, 2, 2, 'ABC University', '2019-2023', 'Business Administration')
-- (3, 3, 1, 'LMN University', '2018-2022', 'Engineering')

----------------------- QUESTIONS ------------------------

Q1: List all employee names.
A1:
```sql
SELECT name FROM employees;
                 
Q2: Who is the employee with the highest salary?
A2:
```sql
SELECT employees.name
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr
ORDER BY salary.salary DESC
LIMIT 1"
```
This SQL query retrieves the name of the employee with the highest salary by joining the employees and salary tables, ordering by salary in descending order, and limiting the result to one row."
"""
})

In [43]:
print(return_CCRMSQL("How many employees have a salary greater than 50,000", context3))

I can help with that! Here is the SQL query to find the number of employees who have a salary greater than 50,000:

```sql
SELECT COUNT(*) 
FROM employees 
JOIN salary ON employees.ID_usr = salary.ID_usr 
WHERE salary.salary > 50000;
```

This query counts the number of employees whose salary is greater than 50,000 by joining the employees and salary tables and filtering for salaries above 50,000.

Sample Result:
| COUNT(*) |
|----------|
| 1        |


# Summary Report



## 🎯 Goal
Test how different prompt styles (`context1`, `context2`, `context3`) affect GPT's SQL response to the question:

> **"How many employees have a salary greater than 50,000?"**

---

## Results Overview

### 1️⃣ Context 1
- **Style:** SQL only.
- **Result:** Correct query, no explanation.
- **Use Case:** Fast and clean automation.

---

### 2️⃣ Context 2
- **Style:** SQL + brief explanation.
- **Result:** Clear, correct, and easy to understand.
- **Use Case:** Learning or documentation.

---

### 3️⃣ Context 3
- **Style:** SQL + explanation + sample result.
- **Result:** Most detailed output with example table.
- **Use Case:** Teaching, demos, or debugging.

---

## 📌 Insights

- All versions returned correct SQL.
- **context1**: Minimal but efficient.
- **context2**: Balanced and readable.
- **context3**: Best for clarity and showcasing output.
- No hallucinations or incorrect logic observed.

---

## ✅ Conclusion

Prompt design significantly shapes output clarity. Use concise prompts for speed, and richer ones for educational or presentation needs.

